# 実験2 電卓（式の計算）

# 2-1 準備

前回の実験で導入したコンマ区切りの数の並びを切り出すパーサcsvを再び取り上げる．

In [ ]:
require("./cumps");
function print(s) { console.log(s); }

var num = pat("[0-9]+");
var csv = sepBy(num, ",");

csv("12,3,456")

parseInt関数を用いると文字列を整数に変換できる

In [ ]:
parseInt("123")

In [ ]:
parseInt("1010101010",2)

In [ ]:
parseInt("ffe8c",16)

In [ ]:
parseInt("123.45")

# 2-2 アクション

パーサの二つの出力（切り出した結果，残りの文字列）のうち前者に接続される関数をパーサのアクションと呼ぶ．例としてparseInt関数をnumパーサのアクションとしてセットするには以下のように代入を用いる．

In [ ]:
num.action = parseInt;

その上でcsv関数を呼び出すと，結果の配列の要素が文字列（例：'12'）ではなく整数（例：12）になっているのが分かる．

In [ ]:
csv("12,3,456")

# 2-3 式の値を求める

与えられた式の値を計算するプログラム（つまり電卓）をつくる．まずは文字列として与えられた式を適切に切り分けるパーサからつくる．

数式の文法（電卓式）
\begin{align*}
\text{<演算子>} &\rightarrow \text{"+"} \mid  \text{"-"} \mid  \text{"*"} \mid  \text{"/"} \\
\text{<式>} &\rightarrow \text{<数>} (\text{<演算子>}　\text{<数>})*
\end{align*}
数式もカツサンドのパターンなのでsepBy関数を用いて簡単にパーサを定義できる．

In [ ]:
var op = oneOf(w("+"), w("-"), w("*"), w("/"));
var expr0 = sepBy(num, op);

In [ ]:
expr0("12+345*6-78/2")

In [ ]:
次に式の計算を（電卓式に前から順に）おこなう関数をつくる．カツサンドのパターンなので，最初の要素を別扱いし，残りを二つずつ組にして考えるとよい．

In [ ]:
function evalExpr(ts) {
    var v = ts[0];
    for (var i = 1; i < ts.length - 1; i += 2) {
        switch (ts[i]) {
          case "+" : v += ts[i + 1]; break;
          case "-" : v -= ts[i + 1]; break;
          case "*" : v *= ts[i + 1]; break;
          case "/" : v = parseInt(v / ts[i + 1]); 
        }
    }
    return v;
}

In [ ]:
最後にこの関数evalExprをパーサexpr0のアクションとしてセットする．

In [ ]:
expr0.action = evalExpr;

In [ ]:
再び先程と同じようにexpr0を呼び出すと今度は式の値が返ってくることが分かる．

In [ ]:
expr0("12+345*6-78/2")

# 2-3 演算の優先順位を考慮する

上記のプログラムは式を電卓式に前から単純に計算するので，普通の数学の式の計算の順序と異なる．例えば1+2*3の値は7になるべきだが9になってしまう．

In [ ]:
expr0("1+2*3")

そこで多項式の考え方を取り入れ，式を項がカツサンドのパターンで並んだものとみなす．項自身もカツサンドのパターンである．
\begin{align*}
\text{<項>} &\rightarrow \text{<数>} ((\text{<"*">}\mid\text{<"/">}) \text{<項>})* \\
\text{<式>} &\rightarrow \text{<項>} ((\text{<"+">}\mid\text{<"-">}) \text{<式>})*
\end{align*}

In [ ]:
これはそのまま以下のパーサ定義に移せる．

In [ ]:
var term   = sepBy(num, oneOf(w("*"), word("/")));
var expr   = sepBy(term, oneOf(w("+"), w("-")));

項の計算も式の計算もそれぞれ電卓式に前から順に行えばよいので，evalExprが全く変更なしに再利用できる（ここがポイント！）

In [ ]:
term.action = expr.action = evalExpr;

今度は値が正しく7になる．

In [ ]:
expr("1+2*3")

# 2-4 式にかっこを許す

明示的にかっこでくくられた式はひとまとまりとみなして計算される．つまりかっこでくくられた式は単一の数と同じ扱いを受ける言わば「数のようなもの」（これを因子（factor）と呼ぶ）である．
\begin{align*}
\text{<因子>} &\rightarrow \text{<数>} \ \mid \ \text{"("}\text{<式>}\text{")"} \\
\text{<項>} &\rightarrow \text{<因子>} ((\text{<"*">}\mid\text{<"/">}) \text{<項>})* \\
\text{<式>} &\rightarrow \text{<項>} ((\text{<"+">}\mid\text{<"-">}) \text{<式>})*
\end{align*}

In [ ]:
この文法もそのまま以下のパーサ定義に移る．ただしexprの定義が再帰的なので最初にダミーの定義が必要（前回の実験参照）．

In [ ]:
function expr(s) { return expr(s); }
var factor = oneOf(num, seq("(", expr, ")"));
var term   = sepBy(factor, oneOf(w("*"), word("/")));
var expr   = sepBy(term, oneOf(w("+"), w("-")));

やはりevalExprが再利用できる．

In [ ]:
term.action = expr.action = evalExpr;

In [ ]:
かっこでかこまれた1+2が先に計算されるので結果は9になる．

In [ ]:
expr("(1+2)*3")

[課題3] termとexprのアクションとしてevalExprの代わりに適当な関数(それをemitASTと呼ぶ)を設定し，数式を読み込むと括弧を補った数式が出力されるようにせよ．ヒント：式の計算でやったように分けて考えると電卓式の単純な処理に帰着する．

In [ ]:
function emitAST(ts) {
    // evalExprを参考にここを自分で書く
}

term.action = expr.action = emitAST;

テスト例（十分ではない．他の例は自分で補え．）

In [ ]:
expr("1+2*3") // ==> (1+(2*3))

In [ ]:
expr("1-2-3-4") // ==> (((1-2)-3)-4)

（参考）文字列の連結には+を使う

In [ ]:
"abc" + "def"